In [1]:
import pandas as pd

df = pd.read_csv('dataset.csv')

In [2]:
df.head(20)

,Unnamed: 0,Fecha,Segmento,hora_llegada,hora_llamado,hora_salida,caja,tienda,status,estado
0,821243,2023-01-01,retail,0.503465,0.503477,0.503581,caja_a,Tienda_A,Atendido,Ciudad de México
1,821244,2023-01-01,retail,0.503581,0.503616,0.511174,caja_a,Tienda_A,Atendido,Ciudad de México
2,821245,2023-01-01,retail,0.503789,0.503801,0.505120,caja_b,Tienda_A,Atendido,Ciudad de México
3,821246,2023-01-01,retail,0.503847,0.505132,0.509333,caja_b,Tienda_A,Atendido,Ciudad de México
4,821247,2023-01-01,retail,0.503894,0.508072,0.509981,caja_c,Tienda_A,Atendido,Ciudad de México
5,821248,2023-01-01,retail,0.505236,0.509333,0.509912,caja_b,Tienda_A,Atendido,Ciudad de México
6,821249,2023-01-01,retail,0.505306,0.509924,0.513512,caja_b,Tienda_A,Atendido,Ciudad de México
7,821250,2023-01-01,retail,0.505363,0.509993,0.511012,caja_c,Tienda_A,Atendido,Ciudad de México
8,821251,2023-01-01,retail,0.505456,0.511012,0.511671,caja_c,Tienda_A,Atendido,Ciudad de México
9,821252,2023-01-01,retail,0.505630,0.511104,0.519993,caja_d,Tienda_A,Atendido,Ciudad de México


In [4]:
len(df)

18510772